This notebook prepares the data for the subsequent notebook `10-Step-Entropy-Analyze.ipynb`, which generates figures illustrating the accumulation of predictive uncertainty in multi-step rollouts, as described in Supplementary Material Section 10.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
num_devices = torch.cuda.device_count()
print("Number of visible GPUs:", num_devices)

for i in range(num_devices):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

current_device = torch.cuda.current_device()
print("Current device index:", current_device)
print("Current device name:", torch.cuda.get_device_name(current_device))

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from allen_cahn_equation import (
    compute_exact_solution_random_ic_vary_Nx,
    visualize_spline_ic,
    plot_both_grids
)

from data_processing import (
    SimpleSerializerSettings,
    scale_2d_array,
    serialize_2d_integers,
    extract_training_and_test
)

from llama_utils import load_model_and_tokenizer, generate_text_multiple

MODEL_NAME = "meta-llama/Llama-3.1-8B"
# MODEL_NAME = "meta-llama/Llama-3.2-3B"
# MODEL_NAME = "meta-llama/Llama-3.2-1B"

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [ ]:
# Define parameters for the Allen-Cahn equation
L = 2       # Length of the spatial domain
k = 0.001   # Thermal diffusivity
T = 0.5     # Total simulation time
Nx = 14     # Number of spatial steps (excluding boundary points)
Nt = 25     # Number of time steps
dx = L/(Nx+1)
dt = T/Nt

# Serialization setup
settings = SimpleSerializerSettings(space_sep=",", time_sep=";")

# Example: Demonstrating the process of generating and visualizing a random initial condition
init_cond_random = np.random.uniform(-0.5, 0.5, size=Nx)
fig = visualize_spline_ic(L, Nx, init_cond_random)
plt.tight_layout()
plt.show()

# Example: Demonstrating how to resample spatial points from an underlying random initial condition
Nx_original = Nx
Nx_new = 14
fig, cs, init_cond_random_new = plot_both_grids(L, Nx_original, Nx_new, init_cond_random)
plt.tight_layout()
plt.show()

In [ ]:
model, tokenizer = load_model_and_tokenizer(MODEL_NAME)

In [ ]:
def generate_multiple_timesteps_with_scores(prompt, model, tokenizer, Nx, number_of_future_predictions):
    """
    Generate multiple future time steps, storing the token probability distributions for each.
    Returns:
        predictions: List of generated text for each time step
        generation_outputs: List of generation outputs containing scores for each time step
    """
    current_prompt = prompt
    if not current_prompt.endswith(";"):
        current_prompt += ";"
    predictions = []
    generation_outputs = []
    for step in range(number_of_future_predictions):
        # Generate next time step
        generated_text, gen_output = generate_text_multiple(
            prompt=current_prompt,
            model=model,
            tokenizer=tokenizer,
            Nx=Nx,
        )
        predictions.append(generated_text.strip())
        generation_outputs.append(gen_output)
        # Update prompt with the generated prediction
        current_prompt += generated_text.strip() + ";"
    
    return predictions, generation_outputs


def calculate_entropies(generation_outputs, Nx):
    """
    Calculate entropy values from generation outputs.
    Returns:
        entropies: Array of shape (Nx, n_future_steps) containing entropy values
        avg_entropy: Average entropy across spatial points
    """
    n_future_steps = len(generation_outputs)
    entropies = np.zeros((Nx, n_future_steps))
    # Calculate entropy for all spatial points
    for time_idx in range(n_future_steps):
        gen_output = generation_outputs[time_idx]
        for grid_idx in range(Nx):
            token_position = grid_idx * 2  # accounting for spatial separators
            if token_position < len(gen_output.scores):
                logits = gen_output.scores[token_position][0]
                p = torch.softmax(logits, -1).clamp_min(1e-30)
                entropy = -(p * torch.log(p)).sum().item()
                entropies[grid_idx, time_idx] = entropy
    # Calculate average entropy across all spatial points
    avg_entropy = entropies.mean(axis=0)
    return entropies, avg_entropy


def analyze_multistep_distributions(model, tokenizer, u_exact_serialized,
                                   input_time_steps, number_of_future_predictions,
                                   Nx, settings):
    """
    Main function to analyze token distributions across multiple future predictions.
    """
    train_serial, test_serial = extract_training_and_test(u_exact_serialized, input_time_steps, settings)
    predictions, generation_outputs = generate_multiple_timesteps_with_scores(
        prompt=train_serial,
        model=model,
        tokenizer=tokenizer,
        Nx=Nx,
        number_of_future_predictions=number_of_future_predictions
    )
    entropies, avg_entropy = calculate_entropies(generation_outputs, Nx)
    
    return predictions, generation_outputs, entropies, avg_entropy

In [ ]:
# Set parameters
input_time_steps = 16
number_of_future_predictions = 10
n_initial_conditions = 20
n_llm_seeds = 20  
all_ic_mean_entropies = []  # mean entropy for each IC
all_ic_mean_avg_entropies = []  # mean average entropy for each IC
all_initial_conditions = []

# Run analysis across multiple initial conditions
for ic_idx in tqdm(range(n_initial_conditions)):
    random.seed(ic_idx)
    np.random.seed(ic_idx)
    torch.manual_seed(ic_idx)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(ic_idx)
    # Generate a new random initial condition for this iteration
    init_cond_random = np.random.uniform(-0.5, 0.5, size=Nx)
    all_initial_conditions.append(init_cond_random)
    # Create spline object for this initial condition
    fig, cs, init_cond_random_new = plot_both_grids(L, Nx_original, Nx_new, init_cond_random)
    plt.close(fig)
    # Compute exact solution for this initial condition
    u_exact = compute_exact_solution_random_ic_vary_Nx(L, k, T, Nx, Nt, spline_obj=cs)
    u_exact_scaled, vmin_exact, vmax_exact = scale_2d_array(u_exact)
    u_exact_serialized = serialize_2d_integers(u_exact_scaled, settings)

    ic_entropies = []
    ic_avg_entropies = []
    
    # Run LLM generation multiple times for this initial condition
    for llm_seed_idx in range(n_llm_seeds):
        random.seed(llm_seed_idx)
        np.random.seed(llm_seed_idx)
        torch.manual_seed(llm_seed_idx)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(llm_seed_idx)
        predictions, generation_outputs, entropies, avg_entropy = analyze_multistep_distributions(
            model=model,
            tokenizer=tokenizer,
            u_exact_serialized=u_exact_serialized,
            input_time_steps=input_time_steps,
            number_of_future_predictions=number_of_future_predictions,
            Nx=Nx,
            settings=settings,
        )
        ic_entropies.append(entropies)
        ic_avg_entropies.append(avg_entropy)
    
    # Average over LLM runs to get point estimate for this initial condition
    ic_entropies = np.array(ic_entropies)
    ic_avg_entropies = np.array(ic_avg_entropies)
    mean_entropy_for_ic = ic_entropies.mean(axis=0)
    mean_avg_entropy_for_ic = ic_avg_entropies.mean(axis=0)
    all_ic_mean_entropies.append(mean_entropy_for_ic)
    all_ic_mean_avg_entropies.append(mean_avg_entropy_for_ic)

# Calculate statistics across initial conditions
all_ic_mean_entropies = np.array(all_ic_mean_entropies)
all_ic_mean_avg_entropies = np.array(all_ic_mean_avg_entropies)
all_initial_conditions = np.array(all_initial_conditions)
mean_entropies = all_ic_mean_entropies.mean(axis=0)
std_entropies = all_ic_mean_entropies.std(axis=0, ddof=1)
mean_avg_entropy = all_ic_mean_avg_entropies.mean(axis=0)
std_avg_entropy = all_ic_mean_avg_entropies.std(axis=0, ddof=1)
se_entropies = std_entropies / np.sqrt(n_initial_conditions)
se_avg_entropy = std_avg_entropy / np.sqrt(n_initial_conditions)

np.savez_compressed(
    "8B_10_step_token_dist.npz",
    mean_entropies_8B=mean_entropies,
    std_entropies_8B=std_entropies,
    se_entropies_8B=se_entropies,
    mean_avg_entropy_8B=mean_avg_entropy,
    std_avg_entropy_8B=std_avg_entropy,
    se_avg_entropy_8B=se_avg_entropy,
    all_initial_conditions=all_initial_conditions,
    all_ic_mean_entropies=all_ic_mean_entropies,
    all_ic_mean_avg_entropies=all_ic_mean_avg_entropies,
    n_initial_conditions=n_initial_conditions,
)